In [3]:
import pandas as pd
from glob import glob
import ahocorasick
from Bio import pairwise2,SeqIO
from tqdm import tqdm

D:\Pythoncode\venv3.7.9\lib\site-packages\Bio\pairwise2.py:283: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  BiopythonDeprecationWarning,


In [4]:
# final_pep_gene_symbol = pd.read_table("/mnt/nfs/wangd/project/paper_db/count/peps_gene_symbol.tsv")
final_pep_gene_symbol = pd.read_table("F:\HPC\paper_db\count\peps_gene_symbol.tsv")

In [5]:
final_pep_gene_symbol

,sequence_x,protein_accessions,peptidoform,gene_symbol
0,AAAAAAAAAPAAAATAATTAATTAATAAQ,GCA_109298,AAAAAAAAAPAAAATAATTAATTAATAAQ,SRP14
1,AAAAPAPPAGPR,GCA_161748,AAAAPAPPAGPR,PLK3
2,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,GCA_60405,AAAGDLGGDHLAFSC[Carbamidomethyl]DVAKEHDVQNTFEEMEK,CBR4
3,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,GCA_60405,AAAGDLGGDHLAFSC[Carbamidomethyl]DVAKEHDVQ[Deam...,CBR4
4,AAAGDLGGDHLAFSCDVAKEHDVQNTFEEMEK,GCA_60405,AAAGDLGGDHLAFSC[Carbamidomethyl]DVAKEHDVQ[Deam...,CBR4
...,...,...,...,...
6203,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVNAC[C...,LYZ
6204,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVN[Dea...,LYZ
6205,YWCNDGKNPGAVNACHLSCSALLQDNIADAVACAK,GCA_170013,YWC[Carbamidomethyl]NDGKN[Deamidated]PGAVNAC[C...,LYZ
6206,YWMHFCGGSLIHPQWVLTAAHCLGPDVK,"GCA_284206,GCA_284208",YWM[Oxidation]HFC[Carbamidomethyl]GGSLIHPQ[Dea...,TPSAB1


In [6]:
all_res_peps = set(final_pep_gene_symbol["sequence_x"])

In [7]:
gene_symbol_proteins = {}
# for record in SeqIO.parse("/mnt/nfs/wangd/project/paper_db/pep_files/save_tmp/Homo_sapiens-GRCh38_all-pep.fa", 'fasta'):
for record in SeqIO.parse("F:\HPC\paper_db\pep_files\save_tmp\Homo_sapiens-GRCh38_all-pep.fa", 'fasta'):
    gene_symbol_list = [x for x in record.description.split(" ") if x.startswith('gene_symbol:')]
    if gene_symbol_list:
        gene_symbol = gene_symbol_list[0].split(':')[-1]
    else:
        continue
    try:
        gene_symbol_proteins[gene_symbol].append(str(record.seq) + ';' + str(record.description))
    except KeyError:
        gene_symbol_proteins[gene_symbol] = [str(record.seq) + ';' + str(record.description)]

In [8]:
seq_gene_symbol_set = set()
seq_no_gene_symbol_set = set()
for index,row in final_pep_gene_symbol.iterrows():
    try:
        gene_symbols = row["gene_symbol"].split(",")
        for gene_symbol in gene_symbols:
            seq_gene_symbol_set.add(row["sequence_x"] + '+' + gene_symbol)
    except AttributeError:
        seq_no_gene_symbol_set.add(row["sequence_x"])

print("There are {} peptides with no corresponding gene symbol found.".format(len(seq_no_gene_symbol_set)))
for seq in seq_no_gene_symbol_set:
    print(seq)

There are 25 peptides with no corresponding gene symbol found.
NTLYHQMNSLRAEDTAVYYCAR
LSCAASGFTFSTYGMHWVR
DLCPLLSEHGLQCSLEPHLNSNLCVYCCK
AEDTAVYFCAK
ASQSISTWLAWYQQKPGK
KTPESFLGPNAALVDLDSLVSRPGPTLPGAK
TGEVVLTQSPGTLSLSPGER
YKSDSDNQQGSGVPSR
SSPVFQIPKNDNIPEQDSLGLSNLQK
LEEECEGREPGLETGTQAADCKDAPLK
ASGYTFTNNGITWVR
ANGGTTDYAAPVK
DTSISTAYMDLSR
ESGPVLVKPTETLTLTCTISGFSLSNAR
SAGGGTYYADSVK
SKANGGTTDYAAPVK
LEWMGWINTGNGNTK
ASGYTYTDYYMHWVR
TPESFLGPNAALVDLDSLVSRPGPTLPGAK
VLIYGASSLQSGVPSR
LLIFGASTR
LSCAVSGFTFSSYAMHWVR
NQVVLTMTNMNPVDTATYYCAR
LEEECEGREPGLETGTQAADCK
NDNIPEQDSLGLSNLQK


In [9]:
gene_symbol_set = set()
for i in seq_gene_symbol_set:
    if i.split("+")[1]:
        gene_symbol_set.add(i.split("+")[1])

In [10]:
len(gene_symbol_set)

1951

In [11]:
blast_infos = dict()
gene_symbol_no_protein_set = set()
for seq_gene_symbol in tqdm(seq_gene_symbol_set):
    pep_seq = seq_gene_symbol.split("+")[0]
    gene_symbol = seq_gene_symbol.split("+")[1]
    if gene_symbol in gene_symbol_proteins.keys():
        for info in gene_symbol_proteins[gene_symbol]:
            prot_seq = info.split(";")[0]
            prot_info = info.split(";")[1]

            alignments_score = pairwise2.align.localms(sequenceA = prot_seq,sequenceB = pep_seq,match = 1,mismatch = -1,open = -1,extend = -1,score_only = True)
            if alignments_score >= len(pep_seq) - 4:
                try:
                    blast_infos[seq_gene_symbol].append(info)
                except KeyError:
                    blast_infos[seq_gene_symbol] = [info]
    else:
        gene_symbol_no_protein_set.add(gene_symbol)
print("There are {} gene_symbols with no corresponding canonical protein sequence found.".format(len(gene_symbol_no_protein_set)))        
for gene_symbol in gene_symbol_no_protein_set:
    print(gene_symbol)

100%|██████████| 3537/3537 [00:12<00:00, 279.53it/s]

There are 14 gene_symbols with no corresponding canonical protein sequence found.
DDX58
UHRF1BP1
FAM207A
FAM160B1
apol1
GBA
srpk1a
H2BS1
WDR92
CBWD2
FAM160B2
EEF1AKNMT
Pwp2
SKIV2L


#### Peptide + gene_symbol + protein

In [12]:
peptide_gene_symbol_protein = dict()
for key, value in blast_infos.items():
    prot_set = set()
    
    pep_seq = key.split("+")[0]
    gene_symbol = key.split("+")[1]
    
    for info in value:
        prot_set.add(info.split(";")[0])

    if peptide_gene_symbol_protein.get(pep_seq):
        peptide_gene_symbol_protein[pep_seq][gene_symbol] = prot_set
    else:
        peptide_gene_symbol_protein[pep_seq] = {gene_symbol:prot_set}

In [13]:
df = pd.DataFrame([(peptide, gene_symbol, ",".join(protein)) for peptide, inner_dict in peptide_gene_symbol_protein.items() for gene_symbol, protein in inner_dict.items()], 
                  columns=['peptide', 'gene_symbol', 'protein'])
# df.to_csv("/mnt/nfs/wangd/project/paper_db/count/blast_canonical_count/peptide_gene_symbol_protein.tsv", header=1, sep="\t", index = None)
df.to_csv(r"F:\HPC\paper_db\count\blast_canonical_count\peptide_gene_symbol_protein1.tsv", header=1, sep="\t", index = None)

In [14]:
df

,peptide,gene_symbol,protein
0,LDNLFLEQEDEEEK,CUL9,MVGERHAGDLMVPLGPRLQAYPEELIRQRPGHDGHPEYLIRWSVLK...
1,FSQPELSEHPQK,PRAM1,MAHHLPAAMESHQDFRSIKAKFQASQPEPSDLPKKPPKPEFGKLKK...
2,AHTTSTGLWGLINNAGHNEVVADAELSPVATFR,HSD11B2,MERWPWPSGGAWLLVAARALLQLLRSDLRLGRPLLAALALLAALDW...
3,TSGLAGEPEGELSKEDHGNTEK,MIA3,XLVPDSKQGKPQSATDYSDPDNVDDGLFIVDIPKTNNDKEVNAEHH...
4,VLDEHIQER,CEP57,MVRHSSSPYVVYPSDKPFLNSDLRRSPSKPTLAYPESNSRAIFSAL...
...,...,...,...
3450,TQLHEDLLPR,PTPRN2,MGPPLPLLLLLLLLLPPRVLPAAPSSVPRGRQLPGRLDGVFGRCQK...
3451,QCQQLQTAIAEVEQR,KRT79,MRSSVSRQTYSTKGGFSSNSASGGSGSQARTSFSSVTVSRSSGSGG...
3452,NLHMAEAPANLEQPTSSFHVGTCFANAQR,LAMA2,MQNLTVEQPIEVKKLFVGGAPPEFQPSPLRNIPPFEGCIWNLVINS...
3453,HPPPPPFQNQQRPPQR,PRR4,MWRGIDSIYPYELLQKNILPAGTCCIYSRVEVFTDVNYEDFTFTIP...


#### Protein + gene_symbol + peptide

In [15]:
protein_gene_symbol_peptide = dict()
for key, value in blast_infos.items():
    pep_seq = key.split("+")[0]
    gene_symbol = key.split("+")[1]
    
    for info in value:
        prot_seq = info.split(";")[0]
        
        if protein_gene_symbol_peptide.get(prot_seq):
            if protein_gene_symbol_peptide.get(prot_seq).get(gene_symbol):
                protein_gene_symbol_peptide[prot_seq][gene_symbol].add(pep_seq)
            else:
                protein_gene_symbol_peptide[prot_seq][gene_symbol] = {pep_seq}
        else:
            protein_gene_symbol_peptide[prot_seq] = {gene_symbol:{pep_seq}} 

In [16]:
df = pd.DataFrame([(protein, gene_symbol, ",".join(peptide)) for protein, inner_dict in protein_gene_symbol_peptide.items() for gene_symbol, peptide in inner_dict.items()], 
                  columns=['protein', 'gene_symbol', 'peptide'])
# df.to_csv("/mnt/nfs/wangd/project/paper_db/count/blast_canonical_count/protein_gene_symbol_peptide.tsv", header=1, sep="\t", index = None)
df.to_csv(r"F:\HPC\paper_db\count\blast_canonical_count\protein_gene_symbol_peptide1.tsv", header=1, sep="\t", index = None)

In [17]:
df

,protein,gene_symbol,peptide
0,MVGERHAGDLMVPLGPRLQAYPEELIRQRPGHDGHPEYLIRWSVLK...,CUL9,LDNLFLEQEDEEEK
1,MVGERHAGDLMVPLGPRLQAYPEELIRQRPGHDGHPEYLIRWSVLK...,CUL9,LDNLFLEQEDEEEK
2,CFPNRLPQLMLQSLSTSEELQRQFHLFQLQRLDKLFLEQEDEEEKR...,CUL9,LDNLFLEQEDEEEK
3,MAHHLPAAMESHQDFRSIKAKFQASQPEPSDLPKKPPKPEFGKLKK...,PRAM1,"FSQPELSEHPQK,KFSQPELSEHPQK"
4,MAHHLPAAMESHQDFRSIKAKFQASQPEPSDLPKKPPKPEFGKLKK...,PRAM1,"FSQPELSEHPQK,KFSQPELSEHPQK"
...,...,...,...
6544,MSDSKEPRVQQLGLLGCLGHGALVLQLLSFMLLAGVLVAILVQVSK...,CLEC4M,NWHNSVTACQEVR
6545,MSDSKEPRVQQLGLLGCLGHGALVLQLLSFMLLAGVLVAILVQVSK...,CLEC4M,NWHNSVTACQEVR
6546,MSDSKEPRVQQLGLLEEDPTTSGIRLFPRDFQFQQIHGHKSSTGCL...,CLEC4M,NWHNSVTACQEVR
6547,MLLVLLSVVLLALSSAQSTDNDVNYEDFTFTIPDVEDSSQRPDQGP...,PRR4,HPPPPPFQNQQRPPQR


#### Gene_symbol + peptide + protein

In [18]:
gene_symbol_peptide_protein = dict()
for key, value in blast_infos.items():
    prot_set = set()
    
    pep_seq = key.split("+")[0]
    gene_symbol = key.split("+")[1]
    
    for info in value:
        prot_set.add(info.split(";")[0])

    if gene_symbol_peptide_protein.get(gene_symbol):
        gene_symbol_peptide_protein[gene_symbol][pep_seq] = prot_set
    else:
        gene_symbol_peptide_protein[gene_symbol] = {pep_seq:prot_set}

In [19]:
gene_symbol_peptide = dict()
gene_symbol_protein = dict()
for gene_symbol,value in gene_symbol_peptide_protein.items():
    for peptide,protein in value.items():
        if gene_symbol_peptide.get(gene_symbol):
            gene_symbol_peptide[gene_symbol].add(peptide)
        else:
            gene_symbol_peptide[gene_symbol] = {peptide}
        
        if gene_symbol_protein.get(gene_symbol):
            gene_symbol_protein[gene_symbol].union(protein)
        else:
            gene_symbol_protein[gene_symbol] = protein
            
gene_symbol_peptide_str = {key: ','.join(value) for key, value in gene_symbol_peptide.items()}
gene_symbol_protein_str = {key: ','.join(value) for key, value in gene_symbol_protein.items()}

In [20]:
df = pd.DataFrame({'gene_symbol': list(gene_symbol_peptide.keys()),
                   'peptide': list(gene_symbol_peptide_str.values()),
                   'protein': list(gene_symbol_protein_str.values())})
# df.to_csv("/mnt/nfs/wangd/project/paper_db/count/blast_canonical_count/gene_symbol_peptide_protein.tsv", header=1, sep="\t", index = None)
df.to_csv(r"F:\HPC\paper_db\count\blast_canonical_count\gene_symbol_peptide_protein1.tsv", header=1, sep="\t", index = None)

In [21]:
df

,gene_symbol,peptide,protein
0,CUL9,LDNLFLEQEDEEEK,MVGERHAGDLMVPLGPRLQAYPEELIRQRPGHDGHPEYLIRWSVLK...
1,PRAM1,"FSQPELSEHPQK,KFSQPELSEHPQK",MAHHLPAAMESHQDFRSIKAKFQASQPEPSDLPKKPPKPEFGKLKK...
2,HSD11B2,AHTTSTGLWGLINNAGHNEVVADAELSPVATFR,MERWPWPSGGAWLLVAARALLQLLRSDLRLGRPLLAALALLAALDW...
3,MIA3,"TSGLAGEPEGELSKEDHGNTEK,DSVEGDALVNGAR,GLVQDETEL...",XLVPDSKQGKPQSATDYSDPDNVDDGLFIVDIPKTNNDKEVNAEHH...
4,CEP57,VLDEHIQER,MVRHSSSPYVVYPSDKPFLNSDLRRSPSKPTLAYPESNSRAIFSAL...
...,...,...,...
1909,DMAC2,LGAAVAPEGSQK,MAAPWASLRLVAPMWNGRIRGIHRLGAAVAPEGNQKKKRTILQFLT...
1910,FGA,VELQDWAGNEAYAEYHFR,MFSMRIVCLVLSVVGTAWTADSGEGDFLAEGGGVRGPRVVERHQSA...
1911,ITPKB,QDSDALPSPELLPQDQDKPFLR,MAVYCYALNSLVIMNSANEMKSGGGPGPSGSETPPPPRRAVLSPGS...
1912,CLEC4M,NWHNSVTACQEVR,MSDSKEPRVQQLGLLEEDPTTSGIRLFPRDFQFQQIHGHKSSTVSK...


#### All Peptides Info Count

In [22]:
all_peptide = set()
all_gene_symbol = set()
all_protein = set()
all_transcript = set()
for key, value in blast_infos.items(): 
    pep_seq = key.split("+")[0]
    all_peptide.add(pep_seq)
    
    gene_symbol = key.split("+")[1]
    all_gene_symbol.add(gene_symbol)
    
    for info in value:
        all_protein.add(info.split(";")[0])
        all_transcript.add([x for x in info.split(" ") if x.startswith('transcript:')][0].split(':')[-1])

In [23]:
print("Number of peptides: {}".format(len(all_peptide)))
print("Number of gene_symbols: {}".format(len(all_gene_symbol)))
print("Number of transcript ids: {}".format(len(all_transcript)))
print("Number of protein seqs: {}".format(len(all_protein)))

Number of peptides: 3413
Number of gene_symbols: 1914
Number of transcript ids: 8400
Number of protein seqs: 6546


### Novel Peptides

In [24]:
fail_peps = all_res_peps - all_peptide

In [131]:
len(fail_peps)

97

In [132]:
df_fail_peps = final_pep_gene_symbol[final_pep_gene_symbol["sequence_x"].isin(fail_peps)]
df_fail_peps

,sequence_x,protein_accessions,peptidoform,gene_symbol
27,AAGFEEIEQDLTQRFEEK,GCA_170981,AAGFEEIEQDLTQRFEEK,DDX58
118,AEDMALYYCAK,GCA_264940,AEDM[Oxidation]ALYYC[Carbamidomethyl]AK,IGHV3-64D
123,AEDTAVYFCAK,GCA_294782,AEDTAVYFC[Carbamidomethyl]AK,NaN
166,AEQQQQQQQQQQQQQQHQHQQQQQQQQQQQQQQQHLSR,GCA_207623,AEQQQQQQQQQQQQQQHQHQQQQQQQQQQQQ[Deamidated]QQQ...,ATXN1
167,AEQQQQQQQQQQQQQQHQHQQQQQQQQQQQQQQQHLSR,GCA_207623,AEQQQQQ[Deamidated]QQQQQQQQQHQHQQQQQQQQQQQ[Dea...,ATXN1
...,...,...,...,...
6088,YFSTSVSWPGRGEPR,GCA_61687,YFSTSVSWPGRGEPR,HLA-C
6122,YIQQQKQQQQQQQQQQQQQQQQQR,GCA_231481,YIQQQKQQQQQQQQQ[Deamidated]QQQQQQQQR,ATXN3
6127,YKSDSDNQQGSGVPSR,GCA_310162,YKSDSDNQQGSGVPSR,NaN
6128,YKSDSDNQQGSGVPSR,GCA_310162,YKSDSDN[Deamidated]QQGSGVPSR,NaN


In [133]:
df_fail_peps_no_gene_symbol = df_fail_peps[df_fail_peps["gene_symbol"].isna()]
df_fail_peps_no_gene_symbol

,sequence_x,protein_accessions,peptidoform,gene_symbol
123,AEDTAVYFCAK,GCA_294782,AEDTAVYFC[Carbamidomethyl]AK,NaN
359,ANGGTTDYAAPVK,GCA_82883,ANGGTTDYAAPVK,NaN
360,ANGGTTDYAAPVK,GCA_82883,AN[Deamidated]GGTTDYAAPVK,NaN
468,ASGYTFTNNGITWVR,GCA_197511,ASGYTFTNNGITWVR,NaN
469,ASGYTYTDYYMHWVR,GCA_152779,ASGYTYTDYYMHWVR,NaN
470,ASGYTYTDYYMHWVR,GCA_152779,ASGYTYTDYYM[Oxidation]HWVR,NaN
491,ASQSISTWLAWYQQKPGK,GCA_213376,ASQSISTWLAWYQQKPGK,NaN
492,ASQSISTWLAWYQQKPGK,GCA_213376,ASQSISTWLAWYQ[Deamidated]Q[Deamidated]KPGK,NaN
493,ASQSISTWLAWYQQKPGK,GCA_213376,ASQ[Deamidated]SISTWLAWYQ[Deamidated]Q[Deamida...,NaN
870,DLCPLLSEHGLQCSLEPHLNSNLCVYCCK,GCA_125675,DLC[Carbamidomethyl]PLLSEHGLQ[Deamidated]C[Car...,NaN


In [135]:
len(set(df_fail_peps_no_gene_symbol["sequence_x"]))

25

In [26]:
auto = ahocorasick.Automaton()

for seq in fail_peps:
    auto.add_word(seq, seq)

auto.make_automaton()

In [27]:
peps_prots = {}
# for prot_file in tqdm(glob('/mnt/nfs/wangd/project/paper_db/pep_files/*.fa')):
for prot_file in tqdm(glob('F:\HPC\paper_db\pep_files\*.fa')):
    prot_seqs = set()
    prots_dict = {}
    parsed_file = SeqIO.parse(prot_file, 'fasta')
    sample_name = prot_file.split('/')[-1].split('-')[1]

    for record in parsed_file:
        try:
            prots_dict[str(record.seq)].append(sample_name + ' ' +  str(record.id) + ' ' +  str(record.seq) + ' ' + str(record.description))
        except KeyError:
            prots_dict[str(record.seq)] = [sample_name + ' ' + str(record.id) + ' ' +  str(record.seq) + ' ' + str(record.description)]
#     print('Total number of unique protein sequences = {} in the fasta file: {}'.format(len(prots_dict.keys()), prot_file))

    #get non-canonical peptides that are found in canonical proteins from ensembl
    found_prots = set()

    for prot_seq in prots_dict.keys():
        for end_ind, found in auto.iter(prot_seq):
            found_prots.add(found)
            try:
                peps_prots[found].append(prots_dict[prot_seq])
            except KeyError:
                peps_prots[found] = [prots_dict[prot_seq]]

# print('Numner of peptides found in the protein files {} and number of peptides {}'.format(len(peps_prots), len(peps)))

100%|██████████| 97/97 [06:28<00:00,  4.00s/it]


In [28]:
sample_peps= {}
gene_ID_peps = {}
protein_ID_peps = {}
protein_Seq_peps = {}
transcript_ID_peps = {}
gene_symbol_peps = {}
peps_infos = {}
res_infos = {}
for pep in tqdm(fail_peps):
    prots = [x.split(' ') for xs in peps_prots[pep] for x in xs]
    samples, proteins, proteins_seq, genes, gene_symbols, transcripts, gene_biotypes = set(), set(), set(), set(), set(), set(), set()

    for prot in prots:
        samples.add(prot[0])
        proteins.add(prot[1])
        proteins_seq.add(prot[2])
        genes.add([x for x in prot if x.startswith('gene:')][0].split(':')[-1])
        transcripts.add([x for x in prot if x.startswith('transcript:')][0].split(':')[-1])
        gene_biotypes.add([x for x in prot if x.startswith('gene_biotype:')][0].split(':')[-1])

        if res_infos.get(prot[2]):
            res_infos[prot[2]][0].add(pep)
            res_infos[prot[2]][1].add(prot[0])
        else:
            res_infos[prot[2]] = [{pep},{prot[0]},set()]

        try:
            gene_symbols.add([x for x in prot if x.startswith('gene_symbol:')][0].split(':')[-1])
            res_infos[prot[2]][2].add([x for x in prot if x.startswith('gene_symbol:')][0].split(':')[-1])
        except IndexError:
            pass

    peps_infos[pep] = [','.join(gene_symbols), ','.join(gene_biotypes), ','.join(samples), ','.join(proteins), ','.join(proteins_seq), ','.join(genes), ','.join(transcripts)]

    for sample in samples:
        try:
            sample_peps[sample].add(pep)
        except KeyError:
            sample_peps[sample] = set()
            sample_peps[sample].add(pep)

    for gene in genes:
        try:
            gene_ID_peps[gene].add(pep)
        except KeyError:
            gene_ID_peps[gene] = set()
            gene_ID_peps[gene].add(pep)

    for prot_id in proteins:
        try:
            protein_ID_peps[prot_id].add(pep)
        except KeyError:
            protein_ID_peps[prot_id] = set()
            protein_ID_peps[prot_id].add(pep)

    for prot_seq in proteins_seq:
        try:
            protein_Seq_peps[prot_seq].add(pep)
        except KeyError:
            protein_Seq_peps[prot_seq] = set()
            protein_Seq_peps[prot_seq].add(pep)

    for transcript in transcripts:
        try:
            transcript_ID_peps[transcript].add(pep)
        except KeyError:
            transcript_ID_peps[transcript] = set()
            transcript_ID_peps[transcript].add(pep)

    for gene_symbol in gene_symbols:
        try:
            gene_symbol_peps[gene_symbol].add(pep)
        except KeyError:
            gene_symbol_peps[gene_symbol] = set()
            gene_symbol_peps[gene_symbol].add(pep)

100%|██████████| 97/97 [00:00<00:00, 769.55it/s]


#### peptide_gca_proteins.tsv

In [29]:
peptide_gca_proteins = pd.DataFrame.from_dict(res_infos, orient='index')
peptide_gca_proteins.columns = ["matching_peptides","sample names","gene_symbol of matching protein"]
peptide_gca_proteins.index.name = "matching_protein_sequence"

# peptide_gca_proteins["peptides_nums"] =  peptide_gca_proteins["matching_peptides"].apply(lambda x: len(x))
# peptide_gca_proteins["sample_nums"] =  peptide_gca_proteins["sample names"].apply(lambda x: len(x))
# peptide_gca_proteins["gene_symbol of matching protein_nums"] =  peptide_gca_proteins["gene_symbol of matching protein"].apply(lambda x: len(x))
peptide_gca_proteins["matching_peptides"] =  peptide_gca_proteins["matching_peptides"].apply(lambda x: ",".join(x))
peptide_gca_proteins["sample names"] =  peptide_gca_proteins["sample names"].apply(lambda x: ",".join(x))
peptide_gca_proteins["gene_symbol of matching protein"] =  peptide_gca_proteins["gene_symbol of matching protein"].apply(lambda x: ",".join(x))

In [30]:
peptide_gca_proteins.to_csv(r"F:\HPC\paper_db\count\blast_canonical_count\peptide_gca_proteins.tsv", header=1, sep="\t")

#### matching_gca_canoical_proteins.tsv

In [89]:
matching_gca_canoical_proteins = pd.read_table(r"F:\HPC\paper_db\count\blast_canonical_count\peptide_gca_proteins.tsv")

In [90]:
matching_gca_canoical_proteins

,matching_protein_sequence,matching_peptides,sample names,gene_symbol of matching protein
0,MKVEACTWAGYSGLYEAIESWDFKKIEKLEEYRLLLKRLQPEFKTR...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58
1,MTTEQRRSLQAFQDYIRKTLDPTYILSYMAPWFREEEVQYIQAEKN...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58
2,MTTEQRRSLQAFQDYIRKTLDPTYILSYMAPWFREGYSGLYEAIES...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58
3,MFSKFTSILQHAVEALAPSLPLQEDFVYHWKAITHYYIETSDDKAP...,FFTGVMEPQLMQTSEMGILTSTALLHR,GCA_018469935.1,FAM160B1
4,MFSKFTSILQHAVEALAPSLPLQEDFVYHWKAITHYYIETSDDKAP...,FFTGVMEPQLMQTSEMGILTSTALLHR,GCA_018469935.1,FAM160B1
...,...,...,...,...
549,MTPGTQSPFFLLLLLTVLTVVTGSGHASSTPGGEKETSATQRSSVP...,CHLGPGHQAGPGLHRPPSPR,"GCA_021950905.1,GCA_018852605.1",MUC1
550,MLCGFQEDATPPPPPRLRRLHRRLPRPEEVGMLSLPPGLLTASGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018471095.1,GCA_018505825.1,GCA_018504085....",BNIP2
551,MLCGFQEDATPPPPPRPRRLHRRLPRPEEVGMLSLPPGLLTVSGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018471065.1,GCA_018503585.1,GCA_018503525....",BNIP2
552,MLCGFQEDATPPPPPRPRRLHRRLPRPEEVGMLSLPPGLLTVSGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018466855.1,GCA_018504365.1,GCA_018469875....",BNIP2


In [104]:
def get_blast_canonical_info(x):
    blast_res = {}
    non_canonical_seq = x["matching_protein_sequence"]

    if gene_symbol_proteins.get(x["gene_symbol of matching protein"]):
        canonical_seqs = gene_symbol_proteins[x["gene_symbol of matching protein"]]

        for canonical_seq in canonical_seqs:
            seq = canonical_seq.split(";")[0]
            alignments = pairwise2.align.globalms(sequenceA = seq,sequenceB = non_canonical_seq, match = 1,mismatch = -1,open = -2,extend = -1,one_alignment_only=True)
            alignment = alignments[0]
            score = alignment.score
            if blast_res:
                for value in blast_res.values():
                    if value.score > score:
                        continue
                    elif value == score:
                        blast_res[canonical_seq] = alignment
                    else:
                        blast_res = {canonical_seq:alignment}
            else:
                blast_res = {canonical_seq:alignment}

        final_res = []
        for key,value in blast_res.items():
            res = []
            i = 0
            j = 0
            for AA1, AA2 in zip(value.seqA, value.seqB):
                i += 1
                j += 1
                if AA1 == AA2:
                    continue
                if AA1 == '-' and AA2 != '-':
                    # insertion
                    res.append(str(i) + "-" + ">" + AA2)
                    i -= 1
                elif AA2 == '-' and AA1 != '-':
                    # deletion
                    res.append(str(i) + AA1 + ">" + "-")
                    j -= 1
                else:
                    # substitution
                    res.append(str(i) + AA1 + ">" + AA2)

            final_res.append({key:res})

        return final_res
    else:
        return ""

In [105]:
tqdm.pandas()
matching_gca_canoical_proteins["match_info"] = matching_gca_canoical_proteins.progress_apply( get_blast_canonical_info,axis=1)

100%|██████████| 554/554 [10:31<00:00,  1.14s/it]


In [112]:
matching_gca_canoical_proteins

,matching_protein_sequence,matching_peptides,sample names,gene_symbol of matching protein,match_info,infonum
0,MKVEACTWAGYSGLYEAIESWDFKKIEKLEEYRLLLKRLQPEFKTR...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58,,0
1,MTTEQRRSLQAFQDYIRKTLDPTYILSYMAPWFREEEVQYIQAEKN...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58,,0
2,MTTEQRRSLQAFQDYIRKTLDPTYILSYMAPWFREGYSGLYEAIES...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58,,0
3,MFSKFTSILQHAVEALAPSLPLQEDFVYHWKAITHYYIETSDDKAP...,FFTGVMEPQLMQTSEMGILTSTALLHR,GCA_018469935.1,FAM160B1,,0
4,MFSKFTSILQHAVEALAPSLPLQEDFVYHWKAITHYYIETSDDKAP...,FFTGVMEPQLMQTSEMGILTSTALLHR,GCA_018469935.1,FAM160B1,,0
...,...,...,...,...,...,...
549,MTPGTQSPFFLLLLLTVLTVVTGSGHASSTPGGEKETSATQRSSVP...,CHLGPGHQAGPGLHRPPSPR,"GCA_021950905.1,GCA_018852605.1",MUC1,{'MTPGTQSPFFLLLLLTVLTVVTGSGHASSTPGGEKETSATQRSS...,1
550,MLCGFQEDATPPPPPRLRRLHRRLPRPEEVGMLSLPPGLLTASGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018471095.1,GCA_018505825.1,GCA_018504085....",BNIP2,{'MEGVELKEEWQDEDFPIPLPEDDSIEADILAITGPEDQPGSLEV...,1
551,MLCGFQEDATPPPPPRPRRLHRRLPRPEEVGMLSLPPGLLTVSGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018471065.1,GCA_018503585.1,GCA_018503525....",BNIP2,{'MEGVELKEEWQDEDFPIPLPEDDSIEADILAITGPEDQPGSLEV...,1
552,MLCGFQEDATPPPPPRPRRLHRRLPRPEEVGMLSLPPGLLTVSGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018466855.1,GCA_018504365.1,GCA_018469875....",BNIP2,{'MEGVELKEEWQDEDFPIPLPEDDSIEADILAITGPEDQPGSLEV...,1


In [107]:
matching_gca_canoical_proteins = matching_gca_canoical_proteins.explode("match_info", ignore_index=True)

In [115]:
matching_gca_canoical_proteins

,matching_protein_sequence,matching_peptides,sample names,gene_symbol of matching protein,match_info,infonum
0,MKVEACTWAGYSGLYEAIESWDFKKIEKLEEYRLLLKRLQPEFKTR...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58,,0
1,MTTEQRRSLQAFQDYIRKTLDPTYILSYMAPWFREEEVQYIQAEKN...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58,,0
2,MTTEQRRSLQAFQDYIRKTLDPTYILSYMAPWFREGYSGLYEAIES...,AAGFEEIEQDLTQRFEEK,"GCA_018469665.1,GCA_018852605.1,GCA_018469875....",DDX58,,0
3,MFSKFTSILQHAVEALAPSLPLQEDFVYHWKAITHYYIETSDDKAP...,FFTGVMEPQLMQTSEMGILTSTALLHR,GCA_018469935.1,FAM160B1,,0
4,MFSKFTSILQHAVEALAPSLPLQEDFVYHWKAITHYYIETSDDKAP...,FFTGVMEPQLMQTSEMGILTSTALLHR,GCA_018469935.1,FAM160B1,,0
...,...,...,...,...,...,...
549,MTPGTQSPFFLLLLLTVLTVVTGSGHASSTPGGEKETSATQRSSVP...,CHLGPGHQAGPGLHRPPSPR,"GCA_021950905.1,GCA_018852605.1",MUC1,{'MTPGTQSPFFLLLLLTVLTVVTGSGHASSTPGGEKETSATQRSS...,1
550,MLCGFQEDATPPPPPRLRRLHRRLPRPEEVGMLSLPPGLLTASGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018471095.1,GCA_018505825.1,GCA_018504085....",BNIP2,{'MEGVELKEEWQDEDFPIPLPEDDSIEADILAITGPEDQPGSLEV...,1
551,MLCGFQEDATPPPPPRPRRLHRRLPRPEEVGMLSLPPGLLTVSGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018471065.1,GCA_018503585.1,GCA_018503525....",BNIP2,{'MEGVELKEEWQDEDFPIPLPEDDSIEADILAITGPEDQPGSLEV...,1
552,MLCGFQEDATPPPPPRPRRLHRRLPRPEEVGMLSLPPGLLTVSGSA...,ASSGDIGSLDYQEFVVDIESR,"GCA_018466855.1,GCA_018504365.1,GCA_018469875....",BNIP2,{'MEGVELKEEWQDEDFPIPLPEDDSIEADILAITGPEDQPGSLEV...,1


In [126]:
matching_gca_canoical_proteins["canonical_protein_seq"] = matching_gca_canoical_proteins["match_info"].apply(lambda x: list(x.keys())[0].split(";")[0] if x and list(x.keys())[0] != "" else None)
matching_gca_canoical_proteins["number_mismatches"] = matching_gca_canoical_proteins["match_info"].apply(lambda x: len(list(x.values())[0]) if x and list(x.values())[0] != "" else None)
matching_gca_canoical_proteins["mismatch_info"] = matching_gca_canoical_proteins["match_info"].apply(lambda x: ",".join(list(x.values())[0]) if x and list(x.values())[0] != "" else None)
matching_gca_canoical_proteins["canonical_protein_ID"] = matching_gca_canoical_proteins["match_info"].apply(lambda x: list(x.keys())[0].split(";")[1].split(" ")[0] if x and list(x.keys())[0] != "" else None)
matching_gca_canoical_proteins["canonical_transcript_ID"] = matching_gca_canoical_proteins["match_info"].apply(lambda x: [x for x in list(x.keys())[0].split(" ") if x.startswith('transcript:')][0].split(':')[-1] if x and list(x.keys())[0] != "" else None)

In [128]:
matching_gca_canoical_proteins = matching_gca_canoical_proteins.drop("match_info", axis=1)

In [130]:
matching_gca_canoical_proteins.to_csv(r"F:\HPC\paper_db\count\blast_canonical_count\matching_gca_canoical_proteins.tsv", header=1, sep="\t",index = None)

#### Novel Peptides Info Count

In [37]:
print("Number of peptides: {}".format(len(fail_peps)))
print("Number of GCA samples matched: {}".format(len(sample_peps)))
print("Number of gene ids matched: {}".format(len(gene_ID_peps)))
print("Number of transcript ids matched: {}".format(len(transcript_ID_peps)))
print("Number of protein ids matched: {}".format(len(protein_ID_peps)))
print("Number of protein seqs matched: {}".format(len(protein_Seq_peps)))
print("Number of gene_symbols matched: {}".format(len(gene_symbol_peps)))

Number of peptides: 97
Number of GCA samples matched: 97
Number of gene ids matched: 1327
Number of transcript ids matched: 3290
Number of protein ids matched: 3290
Number of protein seqs matched: 554
Number of gene_symbols matched: 56


#### Protein_with_two_peptides

In [38]:
protein_with_two_peptides = {}
for protein_seq, peptide_seqs in protein_Seq_peps.items():
    if len(peptide_seqs) > 1:
        protein_with_two_peptides[protein_seq] = ",".join(peptide_seqs)

In [39]:
len(protein_with_two_peptides)

44

In [40]:
df_protein_with_two_peptides = pd.DataFrame(list(protein_with_two_peptides.items()), columns=['protein_seq', 'peptides'])

In [41]:
df_protein_with_two_peptides.to_csv(r"F:\HPC\paper_db\count\blast_canonical_count\protein_with_two_peptides.tsv", header=1, sep="\t",index = None)

In [42]:
df_protein_with_two_peptides

,protein_seq,peptides
0,MGKVRGLRARVHQAAVRPKGEAAPGPAPPAPEATLPPASAAGKDWA...,"DALPELLGLEAGIR,ASPLLAIGQTLAR"
1,MGKVRGLRARVHQAAVRPKGEAAPGPAPPAPEATPPPASAAGKDWA...,"DALPELLGLEAGIR,ASPLLAIGQTLAR"
2,MGKVRGLRARVHQAAVRPKGEAAPGPAPPAPEATPPPASAAGKDWA...,"DALPELLGLEAGIR,ASPLLAIGQTLAR"
3,MGKVRGLRARVHQAAVRPKGEAAPGPAPPAPEATLPPASAAGKDWA...,"DALPELLGLEAGIR,ASPLLAIGQTLAR"
4,MDWTWRILFLVAAATGAHSQVQLVQSGAEVKKPGASVKVSCKASGY...,"DTSISTAYMDLSR,ASGYTYTDYYMHWVR"
5,MSTSSLRRQMKNIVHNYSEAEIKVREATSNDPWGPSSSLMSEIADL...,"KTPESFLGPNAALVDLDSLVSRPGPTLPGAK,TPESFLGPNAALVD..."
6,MSTSSLRRQMKNIVHNYSEAEIKVREATSNDPWGPSSSLMSEIADL...,"KTPESFLGPNAALVDLDSLVSRPGPTLPGAK,TPESFLGPNAALVD..."
7,MGDQSWLWNQAAPGVRSPVFACSVEKGNVPLVLSEHLAHSRDPGSG...,"KTPESFLGPNAALVDLDSLVSRPGPTLPGAK,TPESFLGPNAALVD..."
8,MVFRRFVEVGRVAYVSFGPHAGKLVAIVDVIDQNRALVDGPCTQVR...,"KAPGTKGTAAAAAAAAAAAAAAAK,GTAAAAAAAAAAAAAAAK,GT..."
9,MGCHSSKSTTVAAESQKLEEECEGREPGLETGTQAADCKDAPLKDG...,"LEEECEGREPGLETGTQAADCKDAPLK,LEEECEGREPGLETGTQA..."
